<a href="https://colab.research.google.com/github/LeonardoKoide/projeto_integrado_airbnb/blob/main/projeto_integrado_airbnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pip Installs

In [ ]:
!pip install catboost

# Imports

In [ ]:
import joblib
from google.colab import files
import pandas as pd
import numpy as np
import time
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import time
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor

# Carregando o dataset

In [ ]:
# Carregando o dataset
data = pd.read_csv('/content/Airbnb_Data.csv')

In [ ]:
data.head()

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,...,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds
0,6901257,5.010635,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",3,1.0,Real Bed,strict,True,...,40.696524,-73.991617,Beautiful brownstone 1-bedroom,Brooklyn Heights,2,100.0,https://a0.muscache.com/im/pictures/6d7cbbf7-c...,11201,1.0,1.0
1,6304928,5.129899,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7,1.0,Real Bed,strict,True,...,40.766115,-73.989040,Superb 3BR Apt Located Near Times Square,Hell's Kitchen,6,93.0,https://a0.muscache.com/im/pictures/348a55fe-4...,10019,3.0,3.0
2,7919400,4.976734,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,True,...,40.808110,-73.943756,The Garden Oasis,Harlem,10,92.0,https://a0.muscache.com/im/pictures/6fae5362-9...,10027,1.0,3.0
3,13418779,6.620073,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,flexible,True,...,37.772004,-122.431619,Beautiful Flat in the Heart of SF!,Lower Haight,0,NaN,https://a0.muscache.com/im/pictures/72208dad-9...,94117.0,2.0,2.0
4,3808709,4.744932,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,moderate,True,...,38.925627,-77.034596,Great studio in midtown DC,Columbia Heights,4,40.0,NaN,20009,0.0,1.0


## Explorando o dataset

In [ ]:
# Olhando para as variáveis numéricas

data.select_dtypes(include=[np.number]).describe()

,id,log_price,accommodates,bathrooms,latitude,longitude,number_of_reviews,review_scores_rating,bedrooms,beds
count,7.411100e+04,74111.000000,74111.000000,73911.000000,74111.000000,74111.000000,74111.000000,57389.000000,74020.000000,73980.000000
mean,1.126662e+07,4.782069,3.155146,1.235263,38.445958,-92.397525,20.900568,94.067365,1.265793,1.710868
std,6.081735e+06,0.717394,2.153589,0.582044,3.080167,21.705322,37.828641,7.836556,0.852143,1.254142
min,3.440000e+02,0.000000,1.000000,0.000000,33.338905,-122.511500,0.000000,20.000000,0.000000,0.000000
25%,6.261964e+06,4.317488,2.000000,1.000000,34.127908,-118.342374,1.000000,92.000000,1.000000,1.000000
50%,1.225415e+07,4.709530,2.000000,1.000000,40.662138,-76.996965,6.000000,96.000000,1.000000,1.000000
75%,1.640226e+07,5.220356,4.000000,1.000000,40.746096,-73.954660,23.000000,100.000000,1.000000,2.000000
max,2.123090e+07,7.600402,16.000000,8.000000,42.390437,-70.985047,605.000000,100.000000,10.000000,18.000000


In [ ]:
# Olhando para as variáveis categóricas

data.select_dtypes(include=['object']).describe()

,property_type,room_type,amenities,bed_type,cancellation_policy,city,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,name,neighbourhood,thumbnail_url,zipcode
count,74111,74111,74111,74111,74111,74111,74111,58247,73923,73923,55812,73923,74111,58284,74111,67239,65895,73145
unique,35,3,67122,5,5,6,73479,2554,2,2,80,3087,2,1371,73359,619,65883,769
top,Apartment,Entire home/apt,{},Real Bed,strict,NYC,"Hello, I've been running guest house for Korea...",2017-01-01,t,t,100%,2015-03-30,f,2017-04-30,Bunk bed in the Treat Street Clubhouse,Williamsburg,https://a0.muscache.com/im/pictures/70087089/b...,11211.0
freq,49003,41310,586,72028,32374,32349,8,293,73697,49748,43254,246,54660,1344,8,2862,3,1368


## Checando null values

In [ ]:
# Checando null values
nan_counts = data.isnull().sum().to_frame('nan_count')
total_rows = data.shape[0]

nan_counts['nan_percentage'] = (nan_counts['nan_count'] / total_rows) * 100
nan_counts

,nan_count,nan_percentage
id,0,0.000000
log_price,0,0.000000
property_type,0,0.000000
room_type,0,0.000000
amenities,0,0.000000
accommodates,0,0.000000
bathrooms,200,0.269865
bed_type,0,0.000000
cancellation_policy,0,0.000000
cleaning_fee,0,0.000000


In [ ]:
# Lidando com os null values

data['bathrooms'].fillna(data['bathrooms'].median(), inplace=True)
data['bedrooms'].fillna(data['bedrooms'].median(), inplace=True)
data['beds'].fillna(data['beds'].median(), inplace=True)
data['host_has_profile_pic'].fillna('f', inplace=True)
data['host_identity_verified'].fillna('f', inplace=True)
data['host_response_rate'].fillna('0%', inplace=True)
data['neighbourhood'].fillna('Unknown', inplace=True)
data['review_scores_rating'].fillna(data['review_scores_rating'].median(), inplace=True)
data['thumbnail_url'].fillna('No URL', inplace=True)
data['zipcode'].fillna('Unknown', inplace=True)

# Convertendo de porcentagem para valor numérico
data['host_response_rate'] = data['host_response_rate'].str.replace('%', '').astype(int)


## Visualizando o dataset após as mudanças

In [ ]:
data.select_dtypes(include=[np.number]).describe()

,id,log_price,accommodates,bathrooms,host_response_rate,latitude,longitude,number_of_reviews,review_scores_rating,bedrooms,beds
count,7.411100e+04,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000
mean,1.126662e+07,4.782069,3.155146,1.234628,71.055201,38.445958,-92.397525,20.900568,94.503434,1.265467,1.709611
std,6.081735e+06,0.717394,2.153589,0.581386,43.087044,3.080167,21.705322,37.828641,6.943159,0.851671,1.253389
min,3.440000e+02,0.000000,1.000000,0.000000,0.000000,33.338905,-122.511500,0.000000,20.000000,0.000000,0.000000
25%,6.261964e+06,4.317488,2.000000,1.000000,0.000000,34.127908,-118.342374,1.000000,93.000000,1.000000,1.000000
50%,1.225415e+07,4.709530,2.000000,1.000000,100.000000,40.662138,-76.996965,6.000000,96.000000,1.000000,1.000000
75%,1.640226e+07,5.220356,4.000000,1.000000,100.000000,40.746096,-73.954660,23.000000,99.000000,1.000000,2.000000
max,2.123090e+07,7.600402,16.000000,8.000000,100.000000,42.390437,-70.985047,605.000000,100.000000,10.000000,18.000000


In [ ]:
data.select_dtypes(include=['object']).describe()

,property_type,room_type,amenities,bed_type,cancellation_policy,city,description,first_review,host_has_profile_pic,host_identity_verified,host_since,instant_bookable,last_review,name,neighbourhood,thumbnail_url,zipcode
count,74111,74111,74111,74111,74111,74111,74111,58247,74111,74111,73923,74111,58284,74111,74111,74111,74111
unique,35,3,67122,5,5,6,73479,2554,2,2,3087,2,1371,73359,620,65884,770
top,Apartment,Entire home/apt,{},Real Bed,strict,NYC,"Hello, I've been running guest house for Korea...",2017-01-01,t,t,2015-03-30,f,2017-04-30,Bunk bed in the Treat Street Clubhouse,Unknown,No URL,11211.0
freq,49003,41310,586,72028,32374,32349,8,293,73697,49748,246,54660,1344,8,6872,8216,1368


# Acrescentando Amenities mais relevantes

In [ ]:
# Amenities mais frequentes

boolean_columns = [
    "Wireless Internet", "Kitchen", "Heating", "Essentials", "Smoke detector",
    "Air conditioning", "TV", "Shampoo", "Hangers", "Carbon monoxide detector",
    "Internet", "Laptop friendly workspace", "Hair dryer", "Washer", "Dryer"
]

for amenity in boolean_columns:
    data[amenity] = data['amenities'].str.contains(f'"{amenity}"')

# Convertendo dummies e reduzindo dimensionalidade

In [ ]:
data = pd.get_dummies(data, columns=['property_type', 'room_type', 'bed_type', 'cancellation_policy', 'neighbourhood'], drop_first=True)

In [ ]:
# Verificar e converter tipos de dados
X = data.drop(columns=['log_price']).apply(pd.to_numeric, errors='coerce')
y = data['log_price']

# Exibir os tipos de dados para verificar a conversão
print(X.dtypes)

id                              int64
amenities                     float64
accommodates                    int64
bathrooms                     float64
cleaning_fee                     bool
                               ...   
neighbourhood_Woodlawn           bool
neighbourhood_Woodley Park       bool
neighbourhood_Woodridge          bool
neighbourhood_Woodside           bool
neighbourhood_Wrigleyville       bool
Length: 701, dtype: object


In [ ]:
# Remover colunas não numéricas ou com NaN após a conversão
X_cleaned = X.dropna(axis=1)

# Exibir a nova forma de X para garantir que as colunas não numéricas foram removidas
print(X_cleaned.shape)

(74111, 689)


In [ ]:
# Pipeline para padronizar os dados e aplicar Lasso
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Padronizar os dados
    ('lasso', LassoCV(cv=5, random_state=42))  # Aplicar Lasso com validação cruzada
])

# Iniciar medição de tempo
start_time = time.time()

# Treinar o modelo Lasso
print("Iniciando o treinamento do modelo Lasso...")
pipeline.fit(X_cleaned, y)
print("Treinamento concluído.")

# Identificar as features selecionadas
lasso = pipeline.named_steps['lasso']
selected_features = X_cleaned.columns[(lasso.coef_ != 0)]

# Tempo total de execução
end_time = time.time()
total_time = end_time - start_time

# Exibir as features selecionadas e tempo de execução
print(f"Selected features: {selected_features}")
print(f"Total time: {total_time:.2f} seconds")

# Checar coeficientes e algumas estatísticas
print("Lasso coefficients:")
print(lasso.coef_)
print(f"Number of features selected: {len(selected_features)}")
print(f"Number of non-zero coefficients: {np.sum(lasso.coef_ != 0)}")


Iniciando o treinamento do modelo Lasso...
Treinamento concluído.
Selected features: Index(['id', 'accommodates', 'bathrooms', 'cleaning_fee', 'host_response_rate',
       'latitude', 'longitude', 'number_of_reviews', 'review_scores_rating',
       'bedrooms',
       ...
       'neighbourhood_Winnetka', 'neighbourhood_Winthrop',
       'neighbourhood_Woodhaven', 'neighbourhood_Woodland',
       'neighbourhood_Woodland Hills/Warner Center', 'neighbourhood_Woodlawn',
       'neighbourhood_Woodley Park', 'neighbourhood_Woodridge',
       'neighbourhood_Woodside', 'neighbourhood_Wrigleyville'],
      dtype='object', length=634)
Total time: 41.82 seconds
Lasso coefficients:
[-3.92888021e-06  1.80296306e-01  7.14082755e-02 -2.01135232e-02
 -3.76220191e-02  1.42690239e-02 -3.21312877e-03 -2.33423553e-02
  4.43557003e-02  1.27837726e-01 -4.09863541e-02 -1.28480997e-02
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -7.03717778e-03
  2.53014727e-02  0.00000000e+00  0.00000000e+00  0.00000000e+

# Fazendo teste com vários modelos

In [ ]:
# Função para testar múltiplos modelos de regressão
def evaluate_models(X, y, models):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    results = []

    for name, model in models.items():
        start_time = time.time()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        end_time = time.time()
        elapsed_time = end_time - start_time

        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)

        results.append({
            'Model': name,
            'RMSE': rmse,
            'R2': r2,
            'Time': elapsed_time
        })

        print(f"{name} - RMSE: {rmse:.4f}, R2: {r2:.4f}, Time: {elapsed_time:.2f} seconds")

    return results

# Selecionar os dados com as features relevantes
X_selected = X_cleaned[selected_features]

In [ ]:
# Definir os modelos a serem testados, incluindo o CatBoost
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'ElasticNet Regression': ElasticNet(),
    'XGBoost': XGBRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'CatBoost Regressor': CatBoostRegressor(verbose=0, random_state=42)
}

# Avaliar os modelos
results = evaluate_models(X_selected, y, models)

# Transformar os resultados em um DataFrame para facilitar a visualização
results_df = pd.DataFrame(results)
results_df

Linear Regression - RMSE: 0.4199, R2: 0.6567, Time: 6.55 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=4.55192e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Ridge Regression - RMSE: 0.4197, R2: 0.6571, Time: 1.87 seconds
ElasticNet Regression - RMSE: 0.6369, R2: 0.2103, Time: 2.39 seconds
XGBoost - RMSE: 0.3907, R2: 0.7028, Time: 11.53 seconds
Random Forest - RMSE: 0.3937, R2: 0.6982, Time: 224.40 seconds
Gradient Boosting - RMSE: 0.4168, R2: 0.6618, Time: 71.22 seconds
CatBoost Regressor - RMSE: 0.3841, R2: 0.7129, Time: 16.11 seconds


,Model,RMSE,R2,Time
0,Linear Regression,0.419927,0.656746,6.546286
1,Ridge Regression,0.419692,0.657129,1.872475
2,ElasticNet Regression,0.636933,0.210311,2.390972
3,XGBoost,0.390734,0.702812,11.529659
4,Random Forest,0.393727,0.698241,224.399304
5,Gradient Boosting,0.416808,0.661826,71.215660
6,CatBoost Regressor,0.384060,0.712878,16.106380


# Melhor modelo: Cat Boost.

# Pegando melhores hiperparâmetros para esse modelo

In [ ]:
catboost_model = CatBoostRegressor(verbose=0, random_state=42)

# Definir a grade de hiperparâmetros para buscar
param_distributions = {
    'iterations': [100, 200, 300, 400, 500],
    'depth': [4, 6, 8, 10],
    'learning_rate': [0.01, 0.03, 0.1, 0.3],
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    'bagging_temperature': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0],
    'border_count': [32, 64, 128, 254]
}

random_search = RandomizedSearchCV(
    estimator=catboost_model,
    param_distributions=param_distributions,
    n_iter=50,
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_selected, y)

best_params = random_search.best_params_
print("Best hyperparameters:", best_params)

best_model = random_search.best_estimator_

def evaluate_model(X, y, model, model_name):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    start_time = time.time()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    end_time = time.time()
    elapsed_time = end_time - start_time

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    result = {
        'Model': model_name,
        'RMSE': rmse,
        'R2': r2,
        'Time': elapsed_time
    }

    print(f"{model_name} - RMSE: {rmse:.4f}, R2: {r2:.4f}, Time: {elapsed_time:.2f} seconds")

    return result

# Avaliar o melhor modelo CatBoost
best_catboost_result = evaluate_model(X_selected, y, best_model, 'Best CatBoost Regressor')

print(best_catboost_result)


Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best hyperparameters: {'learning_rate': 0.3, 'l2_leaf_reg': 7, 'iterations': 300, 'depth': 10, 'border_count': 254, 'bagging_temperature': 1.0}
Best CatBoost Regressor - RMSE: 0.3834, R2: 0.7138, Time: 22.64 seconds
{'Model': 'Best CatBoost Regressor', 'RMSE': 0.383420735226009, 'R2': 0.7138328812716511, 'Time': 22.6381516456604}


In [ ]:
best_model = CatBoostRegressor(
    learning_rate=0.3,
    l2_leaf_reg=7,
    iterations=300,
    depth=10,
    border_count=254,
    bagging_temperature=1.0,
    random_state=42
)

best_catboost_result = evaluate_model(X_selected, y, best_model, 'Best CatBoost Regressor')

0:	learn: 0.6106704	total: 65.1ms	remaining: 19.5s
1:	learn: 0.5389850	total: 126ms	remaining: 18.8s
2:	learn: 0.4945756	total: 184ms	remaining: 18.2s
3:	learn: 0.4707552	total: 241ms	remaining: 17.8s
4:	learn: 0.4537662	total: 322ms	remaining: 19s
5:	learn: 0.4396715	total: 380ms	remaining: 18.6s
6:	learn: 0.4316170	total: 438ms	remaining: 18.3s
7:	learn: 0.4256295	total: 495ms	remaining: 18.1s
8:	learn: 0.4223215	total: 557ms	remaining: 18s
9:	learn: 0.4189334	total: 614ms	remaining: 17.8s
10:	learn: 0.4147816	total: 669ms	remaining: 17.6s
11:	learn: 0.4116961	total: 739ms	remaining: 17.7s
12:	learn: 0.4102316	total: 801ms	remaining: 17.7s
13:	learn: 0.4072677	total: 863ms	remaining: 17.6s
14:	learn: 0.4046485	total: 922ms	remaining: 17.5s
15:	learn: 0.4029102	total: 983ms	remaining: 17.5s
16:	learn: 0.4007644	total: 1.05s	remaining: 17.4s
17:	learn: 0.3997974	total: 1.11s	remaining: 17.4s
18:	learn: 0.3991042	total: 1.16s	remaining: 17.2s
19:	learn: 0.3980070	total: 1.22s	remaining:

# Montando um stacking model a partir de três modelos de melhor performance (usando os melhores hiper-parâmetros)

In [ ]:
# Modelos base
estimators = [
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('xgb', XGBRegressor(n_estimators=100, random_state=42)),
    ('catboost', best_model)
]

# Meta-modelo
stacking_model = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())


# Fazendo os testes a partir do modelo persistido

In [ ]:
# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Treinar o modelo de stacking no conjunto de treino
stacking_model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = stacking_model.predict(X_test)

# Avaliar o desempenho do modelo
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Stacking Regressor Test RMSE: {rmse:.4f}")
print(f"Stacking Regressor Test R2: {r2:.4f}")

0:	learn: 0.6106704	total: 58.4ms	remaining: 17.5s
1:	learn: 0.5389850	total: 118ms	remaining: 17.6s
2:	learn: 0.4945756	total: 178ms	remaining: 17.7s
3:	learn: 0.4707552	total: 234ms	remaining: 17.3s
4:	learn: 0.4537662	total: 299ms	remaining: 17.7s
5:	learn: 0.4396715	total: 357ms	remaining: 17.5s
6:	learn: 0.4316170	total: 413ms	remaining: 17.3s
7:	learn: 0.4256295	total: 468ms	remaining: 17.1s
8:	learn: 0.4223215	total: 531ms	remaining: 17.2s
9:	learn: 0.4189334	total: 589ms	remaining: 17.1s
10:	learn: 0.4147816	total: 646ms	remaining: 17s
11:	learn: 0.4116961	total: 707ms	remaining: 17s
12:	learn: 0.4102316	total: 769ms	remaining: 17s
13:	learn: 0.4072677	total: 837ms	remaining: 17.1s
14:	learn: 0.4046485	total: 898ms	remaining: 17.1s
15:	learn: 0.4029102	total: 954ms	remaining: 16.9s
16:	learn: 0.4007644	total: 1.01s	remaining: 16.9s
17:	learn: 0.3997974	total: 1.07s	remaining: 16.7s
18:	learn: 0.3991042	total: 1.13s	remaining: 16.6s
19:	learn: 0.3980070	total: 1.18s	remaining: 1